# Preparation

In a first step, we load or activate the packages.


In [ ]:
library(tidyverse)
library(quanteda)
library(here)
library(openxlsx)
library(flextable)


# Loading and Processing

For this analysis, we will use dta from the following corpora:


* [Australian Radio Talkback](https://www.ausnc.org.au/corpora/art)

* [Griffith Corpus of Spoken Australian English](https://www.ausnc.org.au/corpora/gcsause)

* [Monash corpus](https://www.ausnc.org.au/corpora/monash)

* [The La Trobe Corpus of Spoken Australian English](https://www.ausnc.org.au/corpora/latrobecsause)

## Loading data

The corpora were downloaded and stored in a directory (folder) called `data`. To load the data, we define the paths to the files containing the transcripts (which are located in the `data` folder in the specific sub-directories for the corpora).


In [ ]:
fart <- list.files(here::here("data", "Australian Talkback Radio/files/Raw"), 
                        pattern = ".txt", full.names = T)
fgri <- list.files(here::here("data", "Griffith Corpus of Spoken Australian English/files/Raw"), 
                        pattern = ".txt", full.names = T)
fmon <- list.files(here::here("data", "Monash/files/Text"), 
                        pattern = ".txt", full.names = T)
flat <- list.files(here::here("data", "The La Trobe Corpus of Spoken Australian English/files/Raw"), 
                        pattern = ".txt", full.names = T)


We now check if we have the paths to the data by inspecting the first six paths of files in the *Australian Radio Talkback* corpus.



In [ ]:
# inspect
flextable(head(fart))


We now proceed by loading and processing (cleaning) the data.

### Load ART

We start with the  content of the *Australian Radio Talkback* corpus (art).


In [ ]:
# load raw content
vart <- sapply(fart, function(x){
  x <- readLines(x)
  x <- x[x != ""]
  })
# unlist the object containign the corpus data
arttext <- unlist(vart)
# collapse into a data frame
artdf <- data.frame(names(arttext), names(arttext),arttext) %>%
  # rename columns
  dplyr::rename(corpus = colnames(.)[1],
                file = colnames(.)[2],
                text = colnames(.)[3]) %>%
  # create new columns containing corpus, file, and speaker information as well as a column with clean content
  dplyr::mutate(
    # extract corpus name
    corpus = stringr::str_replace_all(corpus, ".*data/(.*?)/.*", "\\1"),
    # extract file name
    file = stringr::str_replace_all(file, ".*Raw/(.*?)-raw.*", "\\1"),
    # extract speaker
    speaker = stringr::str_replace_all(text, "\\[(.*?)\\].*", "\\1"),
    # extract clean transcript
    textclean = stringr::str_remove_all(text, ".*?\\]"),
    # remove superfluous white spaces
    textclean = stringr::str_squish(textclean))
# remove row names
rownames(artdf) <- NULL
# inspect
flextable(head(artdf))


### Load GRI

We continue with the content of the files of the *Griffith Corpus of Spoken Australian English* (gri).


In [ ]:
vgri <- sapply(fgri, function(x){
  x <- readLines(x, encoding = "UTF-8")
  x <- x[x != ""]
  x <- x[!stringr::str_detect(x, "\\|.*\\|")]
  x <- paste0(x, collapse = " ")
  x <- stringr::str_split(stringr::str_replace_all(x, "( [A-Z]:)", "qwertz\\1"), "qwertz")
  x <- unlist(x)
  x <- stringr::str_squish(x)
})
gritext <- unlist(vgri)
# collapse into df
gridf <- data.frame(names(gritext), names(gritext),gritext) %>%
  dplyr::rename(corpus = colnames(.)[1],
                file = colnames(.)[2],
                text = colnames(.)[3]) %>%
  dplyr::mutate(corpus = stringr::str_replace_all(corpus, ".*data/(.*?)/.*", "\\1"),
                file = stringr::str_replace_all(file, ".*Raw/(.*?)-raw.*", "\\1"),
                speaker = stringr::str_remove_all(text, ":.*"),
                speaker = stringr::str_remove_all(speaker, "\\W.*\\W"),
                speaker = stringr::str_remove_all(speaker, "[^[:alpha:]]"),
                speaker = stringr::str_remove_all(speaker, "[a-z]"),
                textclean = stringr::str_remove_all(text, "<.*?>"),
                textclean = stringr::str_remove_all(textclean, "(.*?)"),
                textclean = stringr::str_remove(textclean, "[0-9]{0,} {0,}[A-Z]{1,}:"),
                textclean = stringr::str_remove_all(textclean, "[^[:alpha:]’ ]"),
                textclean = stringr::str_squish(textclean))
rownames(gridf) <- NULL
# inspect
knitr::kable(head(gridf))


### Load MON

We continue with the content of the files of the *Monash corpus* (mon)


In [ ]:
vmon <- sapply(fmon, function(x){
  x <- readLines(x, encoding = "UTF-8")
  x <- x[x != ""]
  x <- paste0(x, collapse = "qwertz") %>%
  stringr::str_remove_all("qwertz    ") %>%
  stringr::str_split("qwertz") %>%
  unlist() %>%
  stringr::str_squish()
  })
# unlist
montext <- unlist(vmon)
# collapse into df
mondf <- data.frame(names(montext), names(montext), montext) %>%
  dplyr::rename(corpus = colnames(.)[1],
                file = colnames(.)[2],
                text = colnames(.)[3]) %>%
  dplyr::mutate(corpus = stringr::str_replace_all(corpus, ".*data/(.*?)/.*", "\\1"),
                file = stringr::str_replace_all(file, ".*Text/(.*?).txt", "\\1"),
                speaker = paste0("NA"),
                textclean = stringr::str_squish(text))
rownames(mondf) <- NULL
# inspect
knitr::kable(head(mondf))


### Load LAT

We continue with the content of the files of the *The La Trobe Corpus of Spoken Australian English* (lat)


In [ ]:
vlat <- sapply(flat, function(x){
  x <- readLines(x, encoding = "UTF-8")
  x <- x[x != ""]
  })
lattext <- unlist(vlat)
# collapse into df
latdf <- data.frame(names(lattext), names(lattext),lattext) %>%
  dplyr::rename(corpus = colnames(.)[1],
                file = colnames(.)[2],
                text = colnames(.)[3]) %>%
  dplyr::mutate(corpus = stringr::str_replace_all(corpus, ".*data/(.*?)/.*", "\\1"),
                file = stringr::str_replace_all(file, ".*Raw/(.*?)-raw.*", "\\1"),
                speaker = stringr::str_remove_all(text, ":.*"),
                speaker = stringr::str_remove_all(speaker, "\\W.*\\W"),
                speaker = stringr::str_remove_all(speaker, "[^[:alpha:]]"),
                speaker = stringr::str_remove_all(speaker, "[a-z]"),
                textclean = stringr::str_remove_all(text, "^[A-Z]{1,}:{0,1}"),
                textclean = stringr::str_squish(textclean))
rownames(latdf) <- NULL
# inspect
knitr::kable(head(latdf))


# Collapse data sources

We now combine the corpora into a single data frame called *oz*.


In [ ]:
oz <- rbind(artdf, gridf, mondf, latdf)
# inspect
knitr::kable(head(oz))


# Extract utterance-final or

In a next step, we extract utterances with utterance final *or*. We determine this by checking if a string (utterance) ends with the sequence *or* but we allow for another words to come after the or if it has up to three chacraters (e.g., "... or uhm?").


In [ ]:
ufor <- oz %>%
  dplyr::mutate(ufor = ifelse(stringr::str_detect(textclean, " or {0,}.{0,3}$"), 1, 0)) %>%
  dplyr::filter(ufor == 1)
# inspect
knitr::kable(head(ufor$textclean))


In a next step,w e want to extract concordances (keywords-in-context) of potential hits (utterance-final *or*). The context should be two utterances preceding the utterance with utterance-final *or* and two utterances following the instance of utterance-final *or*.



In [ ]:
inds = which(stringr::str_detect(oz$textclean, " or {0,}.{0,3}$"))
# We use lapply() to get all rows for all indices, result is a list
rows <- lapply(inds, function(x) (x-2):(x+2))
# With unlist() you get all relevant rows
ufors <- oz[unlist(rows),]
# insepct
knitr::kable(head(ufors))


We now generate a table with the instances of utterance-final *or* and the preceding as well as subsequent utterances and save the data to out computer for the manual annotation of the functions of utterance-final *or*.



In [ ]:
# label instances
nhits <- sapply(rows, function(x){ length(x) })
nints <- 1:length(rows)
labs <- rep(paste0("instance ", nints), each = nhits)
# label context
contlabs <- rep(c("pre2", "pre1", "hit", "post1", "post2"), length(rows))
# add to kwics
ufors <- ufors %>%
  dplyr::mutate(hit = labs,
                context = contlabs) %>%
  dplyr::select(-speaker, -textclean) %>%
  dplyr::relocate(corpus, file, hit, context, text)


The data frame now contains five lines for each instance: *pre2*, *pre1*, *hit*, *post1*, and *post2*. The instance of utterance-final *or* is shown in the row labeled as *hit*.



In [ ]:
# inspect
knitr::kable(head(ufors))


# Save data to disc

We now save the data so that we can annotate and code the data manually in a spreadsheet software (MS Excel).


In [ ]:
# save
write.xlsx(ufors, here::here("tables", "ufors.xlsx"), sheetName = "Sheet1", 
           col.names = TRUE, 
           row.names = TRUE, 
           append = FALSE)


# Manula Annotation (Coding)

The data (ufors) were annotated manually in a spreadsheet software (MS Excel). 

## UF-or data annotation scheme

The annotation scheme used to code individual instances of utterance-final *or* is provided below. Each instance was inspected and annotated with regard to the categories shown below.

| **Question type**				 | Action type**			       | **Response polarity**		    | **Responses to alternative questions**  | **Explicit-implicit (dis)confirmation**	  | **Response type**			   | **Annotator comment**					       | **Turn-initial particle**
|------------------------------------------------|---------------------------------------------|------------------------------------|-----------------------------------------|-------------------------------------------|----------------------------------------|-------------------------------------------------------------------|-------------------------------------
| Polar question [P]				 | Information-seeking question [Q]	       | Responses to polar questions:	    | First alternative [A]		      | Explicit (yes/no, direct repeat) [E]	  | Type-conforming (yes/no; A or B) [TC]  | Points to potentially follow-up; notes on more complex cases      | um, uh, ah, oh, well, look, mm, no, 
| Alternative question [A]			 | Request (permission-seeking question) [R]   | Yes [Y]			    | Second alternative [B]		      | Implicit [I]				  | Non-type-conforming
| Q-word question [Q]				 | Assertion [A]			       | No [N]				    | Neither [N]
| False positive (i.e. not a question) [FP]	 | Suggestion [S]			       | Yes-no [YN]
|------------------------------------------------|---------------------------------------------|------------------------------------|-----------------------------------------|-------------------------------------------|----------------------------------------|-------------------------------------------------------------------|-------------------------------------											       | Non-answer [NA]



# Data Exploration and Analysis

We now load the manually annotated data.


In [ ]:
ufor_ann <- openxlsx::read.xlsx(here::here("tables", "ufors_annotated.xlsx"), sheet = 1)
# inspect
ufor_ann %>%
  dplyr::filter(corpus == "The La Trobe Corpus of Spoken Australian English") %>%
  head() %>%
  knitr::kable()


You can use and edit the code chunk below to inspect other instances of utterance-final *or* by changing the identifier of the instance, e.g., from `instance 1` (which is the default below) to `instance 51` (overall, there are 98).



In [ ]:
# inspect
ufor_ann %>%
  dplyr::filter(hit == "instance 1") %>%
  knitr::kable()


We will now generate an overview table showing us how frequent different combinations are in the raw data.



In [ ]:
ufor_ann %>%
  dplyr::group_by(hit) %>%
  tidyr::fill(action.type, .direction = "updown") %>%
  tidyr::fill(question.type, .direction = "updown") %>%
  tidyr::fill(response.polarity, .direction = "updown") %>%
  tidyr::fill(`explicit-inferred`, .direction = "updown") %>%
  tidyr::fill(response.type, .direction = "updown") %>%
  dplyr::filter(context == "hit") %>%
  group_by(action.type, question.type, response.polarity, `explicit-inferred`, response.type) %>% 
  dplyr::summarise(Frequency = n()) %>%
  dplyr::arrange(-Frequency) %>%
  knitr::kable()


> It is important to note that there are false positives in the data, i.e. instances that do not really represent instances of utterance-final *or*. Hence, we will remove all instances representing false positives but also non-canonical uses of utterance-final *or* from the data as the analysis will focus on canonical uses of utterance-final *or*. Cononical instances are where the *or* is part of a ploar question.

## Canonical (Q-P) Instances


In [ ]:
# inspect
ufor_can <- ufor_ann %>%
  dplyr::group_by(hit) %>%
  tidyr::fill(action.type, .direction = "updown") %>%
  tidyr::fill(question.type, .direction = "updown") %>%
  tidyr::fill(response.polarity, .direction = "updown") %>%
  tidyr::fill(`explicit-inferred`, .direction = "updown") %>%
  tidyr::fill(response.type, .direction = "updown") %>%
  # filter canonical instances
  dplyr::filter(action.type == "Q" & question.type == "P")
# inspect
knitr::kable(head(ufor_can))


We check how many instances of utterance-final *or* are left.



In [ ]:
length(names(table(ufor_can$hit)))



We are left with 57 canonical instances of utterance-final *or* (i.e. where the utterance containing utterance-final *or* is an information seeking question (Q) and a polar question (P).


We will now check, what instances are left in the data.


In [ ]:
names(table(ufor_can$hit))



You can use and edit the code chunk below to inspect other instances of utterance-final *or* by changing the identifier of the instance, e.g., from `instance 1` (which is the default below) to `instance 9` (overall, there are 57).



In [ ]:
# inspect
ufor_can %>%
  dplyr::filter(hit == "instance 1") %>%
  knitr::kable()


We will now generate an overview table showing us how frequent different combinations are in the canonical data.



In [ ]:
ufor_can %>%
  dplyr::filter(context == "hit") %>%
  group_by(action.type, question.type, response.polarity, `explicit-inferred`, response.type) %>% 
  dplyr::summarise(Frequency = n()) %>%
  dplyr::arrange(-Frequency) %>%
  knitr::kable()


## Canonical with Y or N response

We now want to check the instances where the canonical sequence has received either a positive *yes* [Y] or a negative *no* [N] responses.


In [ ]:
ufor_can_yn <- ufor_can %>%
  dplyr::filter(response.polarity == "Y" | response.polarity == "N")
# inspect
knitr::kable(head(ufor_can_yn))


Again, we will now check, how many instances are left in the data.



In [ ]:
length(names(table(ufor_can_yn$hit)))



We see that there are 46 instances left of canonical sequences where the response is positive or negative.

We will now check, what instances are left in the data.


In [ ]:
names(table(ufor_can_yn$hit))



You can use and edit the code chunk below to inspect other instances of utterance-final *or* with positive and negative responses by changing the identifier of the instance, e.g., from `instance 1` (which is the default below) to `instance 82` (overall, there are 46).



In [ ]:
# inspect
ufor_can_yn %>%
  dplyr::filter(hit == "instance 1") %>%
  knitr::kable()


We will now generate an overview table showing us how frequent different combinations are in the canonical data  with positive and negative responses.



In [ ]:
ufor_can_yn %>%
  dplyr::filter(context == "hit") %>%
  group_by(action.type, question.type, response.polarity, `explicit-inferred`, response.type) %>% 
  dplyr::summarise(Frequency = n()) %>%
  dplyr::arrange(-Frequency) %>%
  knitr::kable()


## Canonical with explicit Y or N response

We now want to check the instances where the canonical sequence has received either an **explicit** positive *yes* [Y] or negative *no* [N] responses.


In [ ]:
ufor_can_eyn <- ufor_can_yn %>%
  dplyr::filter(`explicit-inferred` == "E")
# inspect
knitr::kable(head(ufor_can_eyn))


Again, we will now check, how many instances are left in the data.



In [ ]:
length(names(table(ufor_can_eyn$hit)))



Also, we check, what instances are left in the data.



In [ ]:
names(table(ufor_can_eyn$hit))



You can use and edit the code chunk below to inspect other instances of utterance-final *or* with positive and negative responses by changing the identifier of the instance, e.g., from `instance 1` (which is the default below) to `instance 75` (overall, there are 37).



In [ ]:
# inspect
ufor_can_eyn %>%
  dplyr::filter(hit == "instance 1") %>%
  knitr::kable()


We will now generate an overview table showing us how frequent different combinations are in the canonical data  with explicit positive and negative responses.



In [ ]:
ufor_can_eyn %>%
  dplyr::filter(context == "hit") %>%
  group_by(action.type, question.type, response.polarity, `explicit-inferred`, response.type) %>% 
  dplyr::summarise(Frequency = n()) %>%
  dplyr::arrange(-Frequency) %>%
  knitr::kable()


# Outro



In [ ]:
sessionInfo()

